In [ ]:
                              # import libraries #

In [ ]:
import pandas as pd
import pickle
import numpy as np
import os
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
from keras.utils import np_utils
import random
from keras.preprocessing import image, sequence
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
                          # import paths #

In [ ]:
images_dir = os.listdir("../input/flickr8k/flickr_data/Flickr_Data/")
images_path = '../input/flickr8k/flickr_data/Flickr_Data/Images/'
captions_path = '../input/flickr8k/flickr_data/Flickr_Data/Flickr_TextData/Flickr8k.token.txt'
train_path = '../input/flickr8k/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
val_path = '../input/flickr8k/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.devImages.txt'
test_path = '../input/flickr8k/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt'

captions = open(captions_path, 'r').read().split("\n")
x_train = open(train_path, 'r').read().split("\n")
x_val = open(val_path, 'r').read().split("\n")
x_test = open(test_path, 'r').read().split("\n")

In [ ]:
                                # length of the train #

In [ ]:
len(x_train)

In [ ]:
                             # print the len #

In [ ]:
print(len(captions))
captions[:3]

In [ ]:
tokens = {}

for ix in range(len(captions)-1):
    temp = captions[ix].split("#")
    if temp[0] in tokens:
        tokens[temp[0]].append(temp[1][2:])
    else:
        tokens[temp[0]] = [temp[1][2:]]


In [ ]:
tokens

In [ ]:
from IPython.display import Image, display
temp = captions[100].split("#")
z = Image(filename=images_path+temp[0])
display(z)

for ix in range(len(tokens[temp[0]])):
    print(tokens[temp[0]][ix])


In [ ]:
x_imgs=[]
captions=[]
for img in x_train:
    if img == '':
        continue
    for capt in tokens[img]:
        caption = "<start> "+ capt + " <end>"
        x_imgs.append(img)
        captions.append(caption)

In [ ]:
data={'images':x_imgs,'captions':captions}
df=pd.DataFrame(data)
df.head()

In [ ]:
words = [i.split() for i in captions]
unique = []
for i in words:
    unique.extend(i)
unique = list(set(unique))
print(len(unique))
vocab_size = len(unique)

In [ ]:
word_2_indices = {val:index for index, val in enumerate(unique)}
indices_2_word = {index:val for index, val in enumerate(unique)}

In [ ]:
word_2_indices['UNK'] = 0
indices_2_word[0] = 'UNK'

In [ ]:
with open( "w2i.p", "wb" ) as pickle_f:
    pickle.dump(word_2_indices, pickle_f )
with open( "i2w.p", "wb" ) as pickle_f:
    pickle.dump(indices_2_word, pickle_f )

In [ ]:
with open('../input/img-cap-w2i-i2w/w2i (2).p', 'rb') as f:
    word_2_indices= pickle.load(f, encoding="bytes")
with open('../input/img-cap-w2i-i2w/i2w (2).p', 'rb') as f:
    indices_2_word= pickle.load(f, encoding="bytes")

In [ ]:
vocab_size = len(word_2_indices.keys())
print(vocab_size)

In [ ]:
max_len = 0
for i in captions:
    i = i.split()
    if len(i) > max_len:
        max_len = len(i)

print(max_len)

In [ ]:
padded_sequences, subsequent_words = [], []
for ix in tqdm(range(len(captions))):
    partial_seqs = []
    next_words = []
    text = captions[ix].split()
    text = [word_2_indices[i] for i in text]
    for i in range(1, len(text)):
        partial_seqs.append(text[:i])
        next_words.append(text[i])
    padded_partial_seqs = sequence.pad_sequences(partial_seqs, max_len, padding='post')
    next_words_1hot = np.zeros([len(next_words), vocab_size], dtype=np.bool)
    
    for i,next_word in enumerate(next_words):
        next_words_1hot[i, next_word] = 1
        
    padded_sequences.append(padded_partial_seqs)
    subsequent_words.append(next_words_1hot)

In [ ]:
model = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')
#model.summary()

In [ ]:
def preprocessing(img_path):
    im = image.load_img(img_path, target_size=(224,224,3))
    im = image.img_to_array(im)
    im = np.expand_dims(im, axis=0)
    return im

In [ ]:
imgs={}
for i in tqdm(x_train[:6000]):
    if i in imgs.keys():
        continue
    path = images_path + i
    img = preprocessing(path)
    pred = model.predict(img).reshape(2048)
    imgs[i]=pred

In [ ]:
len(imgs)

In [ ]:
with open( "proces_imgs.p", "wb" ) as pickle_f:
    pickle.dump(imgs, pickle_f )

In [ ]:
with open('../input/img-cap-63-acc-model/proces_imgs.p', 'rb') as f:
    imgs= pickle.load(f, encoding="bytes")

In [ ]:
img = []
for i in tqdm(range(df.shape[0])):
    if df.iloc[i, 0] in imgs.keys():
        img.append(imgs[df.iloc[i, 0]])

img = np.asarray(img)
print(img.shape)

In [ ]:
embedding_size = 128
image_model = Sequential()
image_model.add(Dense(embedding_size, input_shape=(2048,), activation='relu'))
image_model.add(RepeatVector(max_len))

image_model.summary()

In [ ]:
language_model = Sequential()
language_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
language_model.add(LSTM(256, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_size)))

language_model.summary()

In [ ]:
conca = Concatenate()([image_model.output, language_model.output])
x = LSTM(128, return_sequences=True)(conca)
x = LSTM(512, return_sequences=False)(x)
x = Dense(vocab_size)(x)
out = Activation('softmax')(x)
model = Model(inputs=[image_model.input, language_model.input], outputs = out)
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

In [ ]:
model.load_weights('../input/img-cap-63-acc-model/best_model_acc (1).h5')

In [ ]:
def tr_gen(inp,ne_words,batch_size=256):
    cap=inp[1]
    imgs=inp[0]
    while True:
        for i in range(0,30000-batch_size,batch_size):
            n_words=[]
            i_cap=[]
            im=[]
            for j in range(batch_size):
                n_words.extend(ne_words[i+j].tolist())
                i_cap.extend(cap[i+j].tolist())
            for ix in range(i,i+batch_size):
                for iy in range(cap[ix].shape[0]):
                    im.append(imgs[ix])
                
            i_cap=np.array(i_cap)
            n_words=np.asarray(n_words)
            im= np.asarray(im)
        
            yield [im,i_cap],n_words

In [ ]:
train_gen=tr_gen([img,padded_sequences],subsequent_words,batch_size=256)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('best_model_acc.h5', monitor = 'accuracy' , mode = 'max', verbose = 1 , save_best_only = True)

In [ ]:
hist=model.fit_generator(train_gen,steps_per_epoch=30000//256,callbacks = [mc],epochs=100)

In [ ]:
def preprocessing(img_path):
    im = image.load_img(img_path, target_size=(224,224,3))
    im = image.img_to_array(im)
    im = np.expand_dims(im, axis=0)
    return im

In [ ]:
def get_encoding(model, img):
    image = preprocessing(img)
    pred = model.predict(image).reshape(2048)
    return pred

In [ ]:
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')

In [ ]:
l=os.listdir('../input/flickr8k/flickr_data/Flickr_Data/Images/')
l[:2]

In [ ]:
img = '../input/flickr8k/flickr_data/Flickr_Data/Images/'+l[4681]
#img='../input/backgrd-sample-imgs/2019-02-10-13-40-01-855.jpg'
test_img = get_encoding(resnet, img)

In [ ]:
def predict_captions(image):
    start_word = ["<start>"]
    while True:
        par_caps = [word_2_indices[i] for i in start_word]
        par_caps = sequence.pad_sequences([par_caps], maxlen=max_len, padding='post')
        preds = model.predict([np.array([image]), np.array(par_caps)])
        word_pred = indices_2_word[np.argmax(preds[0])]
        start_word.append(word_pred)
        
        if word_pred == "<end>" or len(start_word) > max_len:
            break
            
    return ' '.join(start_word[1:-1])

Argmax_Search = predict_captions(test_img)
z = Image(filename=img)
display(z)

print(Argmax_Search)